In [10]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6
println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=2
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p]
f=sum(vecf)

g=[1.0-sum(x[I[j]].^2) for j in 1:p]
J=[j:j for j in 1:p]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

randx=2*rand(Float64,n).-1
for j in 1:p
    randx[I[j]]=randx[I[j]]./norm(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=4
Number of equality constraints: l=1


In [11]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [12]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=2

println("Term sparse order: t=",t)

Relaxed order: k=2
Term sparse order: t=2


In [13]:
using TSSOS


@time begin
    opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);
    #opt,sol,data=cs_tssos_higher!(data,TS="block");
end

------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 53              
  Cones                  : 0               
  Scalar variables       : 12              
  Matrix variables       : 13              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

(-1.110824066253166, nothing, TSSOS.mdata_type(6, 0, 5, [2, 2, 2, 2, 2], SparseMatrixCSC{UInt8,UInt32}[
  [1,  1]  =  0x02
  [1,  2]  =  0x01
  [2,  2]  =  0x01
  [2,  3]  =  0x02
  [2,  4]  =  0x01
  [3,  4]  =  0x01
  [2,  5]  =  0x01
  [4,  5]  =  0x01
  [3,  6]  =  0x02
  [3,  7]  =  0x01
  [4,  7]  =  0x01
  [4,  8]  =  0x02
  [4,  9]  =  0x01
  [5,  9]  =  0x01
  [4, 10]  =  0x01
  [6, 10]  =  0x01
  [5, 11]  =  0x02
  [5, 12]  =  0x01
  [6, 12]  =  0x01
  [6, 13]  =  0x02, 
  [1, 1]  =  0x02
  [2, 2]  =  0x02, 
  [2, 1]  =  0x02
  [3, 2]  =  0x02
  [4, 3]  =  0x02, 
  [4, 1]  =  0x02
  [5, 2]  =  0x02
  [6, 3]  =  0x02, 
  [6, 1]  =  0x02, 
  [6, 1]  =  0x02], Array{Float64,1}[[0.35072651389979903, -0.4872598815355609, 0.7006800500765595, 0.11781351773757365, -0.6963440465881354, 0.29481242391506823, 0.2061806929536676, -0.4310534205437757, -0.8844025265748674, 0.34388368932222324, -0.7288432486166831, -0.11052304217916031, 0.7533280371342874], [-1.0, -1.0, 1.0], [-1.0, -1.0, -1

In [14]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[3, 2]
[2, 1]
------------------------------------------------------
  Number of cliques: p=3
  Largest clique size: u=3


  Largest constant trace: a=5.0
  Number of blocks: omega=14
  Number of equality consraints: zeta=67
  Size of the largest block: s^max=7
Modeling time:
  1.146710 seconds (1.13 M allocations: 55.738 MiB)
iter=1   val=-6.1323134014158445   gap=1.2452879868175896e-16   feas=10.505901273097955
iter=2   val=-2.044104467138615   gap=0.7077432556421772   feas=2.6995821751563978
iter=4   val=-2.158603544061307   gap=0.3422625284105917   feas=3.0396414231888222
iter=8   val=-1.8478707393084481   gap=0.1524507792421174   feas=1.9400971043613255
iter=16   val=-1.053157729598648   gap=0.3775014303058916   feas=0.482886185627037
iter=32   val=-1.0577759923878125   gap=0.22759140386125182   feas=0.3122255879866713
iter=64   val=-1.1641103478107366   gap=0.20250782505778334   feas=0.21106239487872464
iter=128   val=-1.1582200479926295   gap=0.10475586647852297   feas=0.10992400457028083
iter=256   val=-1.1129245291761796   gap=0.053269479298382126   feas=0.05370057656413033
iter=512   val=-1.10264

-1.111642752663142

In [15]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:       23

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        4
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0